In [ ]:
!pip install openai
from openai import OpenAI
import openai
import pandas as pd
import re
from time import sleep
import numpy as np
import json

#openAI api phishing analysis
def openAI():
  print("OpenAI\n")
  #get api key
  input_file = 'OpenAIAPIKey.json'
  with open(input_file, 'r') as file:
      data = json.load(file)
  key = data['key']

  df = pd.read_csv("test_data_openai.csv")

  #set up api connection
  api_key = key
  client = OpenAI(api_key=api_key)

  labels = []
  emails = ""
  counter = 0
  expected = ""
  for i in range(len(df)):
    #Phishing_Email
    email = str(df['Email Text'][i])
    labels.append(df['Email Type'][i])

    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          {"role": "user", "content": f"Determine if this email is phishing or safe. If an email is phishing respond with a 1, if it is safe respond with a 0.  Do not use any words. Here it is: {email}"}
      ]
    )
    expected += completion.choices[0].message.content
    emails = ""


  expectarray = []
  for char in expected:
      if char == '1':
          expectarray.append(1)
      elif char == '0':
          expectarray.append(0)

  actual = []
  for i in range(len(labels)):
    label = labels[i]
    if(label == "Safe Email"):
      actual.append(0)
    else:
      actual.append(1)

  true_positive = 0
  false_positive = 0
  true_negative = 0
  false_negative = 0
  for i in range(len(actual)):
    try:
      if actual[i] == expectarray[i]:
          if actual[i] == 1:
              true_positive += 1
          else:
              true_negative += 1
      else:
          if actual[i] == 1:
              false_negative += 1
          else:
              false_positive += 1
    except IndexError:
      print(i)

  confusion_matrix = np.array([[true_negative, false_positive],
                                [false_negative, true_positive]])

  print("Confusion Matrix:")
  print(confusion_matrix)

  accuracy2 = (true_positive + true_negative) / len(actual) if len(actual) > 0 else 0
  precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
  recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0

  print("Accuracy:", accuracy2)
  print("Precision:", precision)
  print("Recall:", recall)


In [ ]:
def main():
  openAI()

if __name__ == "__main__":
  main()

OpenAI

6970
6971
Confusion Matrix:
[[1329 2156]
 [1084 2401]]
Accuracy: 0.5349971313826736
Precision: 0.5268817204301075
Recall: 0.6889526542324247
